In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
# 1. Configuration
DATA_PATH = r"C:\mask_data"  # Your C drive path
IMG_SIZE = 224
classes = ['mask', 'nomask']

def load_data_manually(path):
    data = []
    labels = []
    for category in classes:
        folder_path = os.path.join(path, category)
        label = classes.index(category)
        for img_name in os.listdir(folder_path):
            try:
                img_path = os.path.join(folder_path, img_name)
                # cv2 is more robust against weird Windows filenames
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                data.append(image)
                labels.append(label)
            except Exception as e:
                print(f"Skipping broken file: {img_name}")
                continue
    return np.array(data) / 255.0, np.array(labels)

print("Loading images manually (this skips the Unicode error)...")
X, y = load_data_manually(DATA_PATH)
print(f"Successfully loaded {len(X)} images!")


Loading images manually (this skips the Unicode error)...
Skipping broken file: 0_0_œ¬‘ÿ.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-23 132115.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-23 132400.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 171804.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 172039.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 202509.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 205216.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 215234.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 215615.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 220536.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 222124.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 224833.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 225329.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-24 225427.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-25 150422.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-25 150847.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-25 150921.png
Skipping broken file: 0_0_≈˙◊¢ 2020-02-25 185823.png
Skippi

In [4]:
# 2. Build Model
base_model = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# 3. Training
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=32)

Epoch 1/10


96/96 [==============================] - 48s 416ms/step - loss: 0.1004 - accuracy: 0.9637 - val_loss: 0.2172 - val_accuracy: 0.9268
Epoch 2/10
96/96 [==============================] - 37s 388ms/step - loss: 0.0442 - accuracy: 0.9866 - val_loss: 0.2426 - val_accuracy: 0.9124
Epoch 3/10
96/96 [==============================] - 37s 390ms/step - loss: 0.0264 - accuracy: 0.9908 - val_loss: 0.2609 - val_accuracy: 0.9281
Epoch 4/10
96/96 [==============================] - 37s 389ms/step - loss: 0.0219 - accuracy: 0.9925 - val_loss: 0.2092 - val_accuracy: 0.9386
Epoch 5/10
96/96 [==============================] - 37s 390ms/step - loss: 0.0158 - accuracy: 0.9944 - val_loss: 0.2692 - val_accuracy: 0.9268
Epoch 6/10
96/96 [==============================] - 37s 388ms/step - loss: 0.0114 - accuracy: 0.9957 - val_loss: 0.2541 - val_accuracy: 0.9307
Epoch 7/10
96/96 [==============================] - 37s 388ms/step - loss: 0.0118 - accuracy: 0.9951 - val_loss: 0.3508 - val_accuracy: 0.91

In [6]:
# Save the model so we don't have to train it again
model.save("mask_detector_model.h5")
print("Model saved as mask_detector_model.h5!")

C:\Users\Ilango\AppData\Local\anaconda3\envs\tf_env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as mask_detector_model.h5!


In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [8]:
# 1. Load your trained model
model = load_model("mask_detector_model.h5")

In [9]:
# 2. Use OpenCV's built-in face detector
face_clsfr = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [21]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# 1. Load your model
model = load_model("mask_detector_model.h5")
face_clsfr = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

# 2. Reset Camera with specific width/height
source = cv2.VideoCapture(0, cv2.CAP_DSHOW)
source.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
source.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Starting Camera... If screen is black, try waving your hand.")

while True:
    ret, img = source.read()
    
    # If the frame is empty, we skip the rest of the loop
    if not ret or img is None:
        continue 
    
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.1, 5, minSize=(80, 80))  

    for (x, y, w, h) in faces:
        face_img = img[y:y+h, x:x+w]
        resized = cv2.resize(face_img, (224, 224))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 224, 224, 3))
        
        prediction = model.predict(reshaped, verbose=0)
        prob = prediction[0][0]
        
        # Determine Label
        label = 1 if prob > 0.5 else 0 
        text = "NO MASK" if label == 1 else "MASK"
        color = (0, 0, 255) if label == 1 else (0, 255, 0)

        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Force the window to show something even if no face is found
    cv2.imshow('Face Mask Detector', img)
    
    # Use waitKey(10) instead of 1 to give the CPU more time to draw the frame
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

source.release()
cv2.destroyAllWindows()

Starting Camera... If screen is black, try waving your hand.
